## Builds all our models x-validated


In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from sklearn.cluster import KMeans
from StringIO import StringIO
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn import cross_validation, grid_search
from sklearn.ensemble import RandomForestRegressor


from vectorizing_funcs import *
from modeling_funcs import *


In [2]:
df = pd.read_csv('../all_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col="SubjectID")
slope.index = slope.index.astype(str)

print "df: ", df.shape, df.SubjectID.unique().size
print "slope: ", slope.shape, slope.index.unique().size
display(df.head(2))
display(slope.head(2))

df:  (1642927, 6) 2424
slope:  (2424, 1) 2424


,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,Demographic,Gender,F,NaN,0.0
1,533,Demographic,Age,65,NaN,0.0


,ALSFRS_slope
SubjectID,
533,-0.965608
649,-0.921717


In [3]:
def apply_on_test(test_data, all_feature_metadata, train_data_means, train_data_std, 
                 clustering_columns, bins, forest, best_features_per_cluster, model_per_cluster):
    
    # Vectorizing
    vectorized, _ = vectorize(test_data, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    
    print "applying on: ", normalized.shape
    
    # Clustering
    
    for_clustering = normalized
    clusters = pd.DataFrame(index = for_clustering.index.astype(str))
    clusters['cluster'] = np.digitize(forest.predict(for_clustering), bins)
    print "applied cluster cnt: ", np.bincount(clusters.cluster)

    X = normalized.join(clusters)
    
    buf = filter_only_selected_features(test_data.set_index("SubjectID"), clusters, \
                                        best_features_per_cluster)    
    s_df = pd.read_csv(StringIO(buf), sep='|', index_col=False, dtype='unicode')
    s_vectorized, _ = vectorize(s_df, all_feature_metadata)
    s_normalized, _ = normalize(s_vectorized, all_feature_metadata, train_data_means, train_data_std)    
    input_for_model = s_normalized.join(clusters)    
    
    pred = input_for_model.apply(apply_model, args=[model_per_cluster], axis = 1)
    return input_for_model, pred
    

In [4]:
def train_it(train_data, my_n_clusters):
        global ts_funcs_to_features
        # Prepare metadata
        ts_funcs_to_features = add_frequent_lab_tests_to_ts_features(train_data, ts_funcs_to_features)
        all_feature_metadata = invert_func_to_features(ts_funcs_to_features, "ts")
        all_feature_metadata.update(invert_func_to_features(dummy_funcs_to_features, "dummy"))
        all_feature_metadata = learn_to_dummies_model(train_data, all_feature_metadata)
        
        # Vectorizing
        vectorized, all_feature_metadata = vectorize(train_data, all_feature_metadata)
        train_data_means = vectorized.mean()
        train_data_std = vectorized.std()            
        normalized, all_feature_metadata = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)

        everybody = normalized.join(slope)
        X = everybody.drop(['ALSFRS_slope'], 1)
        Y = everybody[['ALSFRS_slope']]
        print "train_data: ", X.shape, Y.shape
        
        # Clustering
        #for_clustering = normalized[clustering_columns]
        #kmeans = KMeans(init='k-means++', n_clusters=my_n_clusters)
        #clusters['cluster'] = kmeans.fit_predict(for_clustering)

        forest = grid_search.GridSearchCV(RandomForestRegressor(min_samples_leaf=60, min_samples_split=260, random_state=0), 
                               {'min_samples_leaf': range(60,61,10), 'n_estimators': [1000]})
        forest.fit(X, Y.ALSFRS_slope)
        bins = np.percentile(forest.predict(X), range(20,100,20))
                          
        # Note we must convert to str to join with slope later
        clusters = pd.DataFrame(index = normalized.index.astype(str))
        clusters['cluster'] = np.digitize(forest.predict(X), bins)
        print "train cluster cnt: ", np.bincount(clusters.cluster)

        X = X.join(clusters)
        Y = Y.join(clusters)

        best_features_per_cluster = stepwise_best_features_per_cluster(X, Y, all_feature_metadata)
        print "best_features_per_cluster: ", best_features_per_cluster 
        buf = filter_only_selected_features(train_data.set_index("SubjectID"), clusters, \
                                            best_features_per_cluster)

        s_df = pd.read_csv(StringIO(buf), sep='|', index_col=False, dtype='unicode')
        s_vectorized, _ = vectorize(s_df, all_feature_metadata)
        s_normalized, _ = normalize(s_vectorized, all_feature_metadata, train_data_means, train_data_std)    
        s_X = s_normalized.join(clusters)
        
        model_per_cluster = get_model_per_cluster(s_X, Y)
        
        return all_feature_metadata, train_data_means, train_data_std, \
                     bins, forest, best_features_per_cluster, model_per_cluster


In [7]:
from datetime import datetime

def train_and_test(df, slope, my_n_clusters=2):
    kf = KFold(df.SubjectID.unique().size, n_folds=3)
    fold, test_rmse, train_rmse = 0, 0.0, 0.0

    for train, test in kf:
        train_data = df[df.SubjectID.isin(df.SubjectID.unique()[train])]
        test_data = df[df.SubjectID.isin(df.SubjectID.unique()[test])]
        print
        print "*"*30
        print "fold: %d" % fold
        tick = datetime.now()
        
        all_feature_metadata, train_data_means, train_data_std, \
                     bins, forest, best_features_per_cluster, model_per_cluster = train_it(train_data, my_n_clusters)

        input_for_model, pred = apply_on_test(train_data, all_feature_metadata, train_data_means, train_data_std, 
                     clustering_columns, bins, forest, best_features_per_cluster, model_per_cluster)
        res = pred.join(slope)
        fold_train_rmse = np.sqrt(np.mean((res.prediction - res.ALSFRS_slope) ** 2))

        input_for_model, pred = apply_on_test(test_data, all_feature_metadata, train_data_means, train_data_std, 
                     clustering_columns, bins, forest, best_features_per_cluster, model_per_cluster)
        res = pred.join(slope)
        fold_test_rmse += np.sqrt(np.mean((res.prediction - res.ALSFRS_slope) ** 2))
        
        input_for_model.to_csv('../x_results/test_%d_input_for_model.csv' % fold,sep='|')
        res.to_csv('../x_results/test_%d_prediction.csv' % fold,sep='|')

        fold += 1
        print "fold RMS Error train, test: ", fold_train_rmse, fold_test_rmse
        train_rmse += fold_train_rmse
        test_rmse += fold_test_rmse

        tock = datetime.now()   
        diff = tock - tick 
        print "minutes for fold: ", diff.seconds / 60

            
    print "X-validated RMS Error train, test: ", train_rmse / kf.n_folds, test_rmse / kf.n_folds



In [8]:
for n_clusters in range(2, 4):
    print "*"*60
    print "*"*60
    train_and_test(df, slope, n_clusters)

************************************************************
************************************************************

******************************
fold: 0
train_data:  (1616, 145) (1616, 1)
train cluster cnt:  [323 323 323 323 324]
cluster: 0 with size: (323, 146) with mean target: -1.18324478115 std: 0.765767251086
best we can do with all features: 0.756029078052
adding best family: ('respiratory', 0.74864553066423289) time: 1.84099984169
adding best family: ('Monocytes', 0.73517132383034056) time: 2.04699993134
adding best family: ('Race', 0.72082989004365416) time: 2.51699995995
adding best family: ('weight', 0.71184367142664862) time: 3.38100004196
adding best family: ('Total Cholesterol', 0.70472369303363758) time: 10.503000021
adding best family: ('Alkaline Phosphatase', 0.69574616673062428) time: 12.5180001259
cluster: 1 with size: (323, 146) with mean target: -0.85739851173 std: 0.620179903326
best we can do with all features: 0.605698508808
adding best family: ('Total C

UnboundLocalError: local variable 'fold_test_rmse' referenced before assignment